# Test Cases



In [1]:

import sys
from mylist import *
from mycrm import *
from myserver import *
from myvm import *

from inspect import currentframe, getframeinfo

# test.py for toy cloud assignment.
# feel free to comment the relevant parts in main function.

## test helper functions

global test_no
global crm
test_no=0
def mycheck(result, msg, lno=0):
    global test_no
    test_no += 1
    cf=currentframe()
    l1=cf.f_back.f_lineno
    if lno:
        l=f'{lno},{l1}'
    else:
        l=f'{l1}'
    if result:
        print(f'{l} tno: {test_no} {msg} PASSED ')
    else:
        print(f'{l}: tno: {test_no} {msg} FAILED ')
        sys.exit(1)

# template crmstat to copy and change values.
#mys=crm_stat(stotal=1,sidle=1,sactive=0,sfail=0,vtotal=1,vprovision=1,vactive=0,'vidle', vdeleted=0)
#print(mys)


def check_stat(s1):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_stat()
    mycheck((s==s1),f'{s1} == {s}', lno)

def check_server_count(count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_servers(state)
    mycheck(len(s) == count, f'server_count {len(s)} == {count} for state {state}', lno)

    s=crm.get_servers('any')
    c=0
    for i in s:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c == count, f'server_count {c} == {count} for state {state}', lno)

def check_server_state(sname, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    assert(state != 'any')
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sname and i.state == state:
            f=1
            break
    mycheck(f==1, f'server {sname} with state {state} found', lno)
    s=crm.get_servers(state)
    f=0
    for i in s:
        if i.sc.name == sname and i.state == state:
            f=1
            break
    mycheck(f==1, f'server {sname} with state {state} found', lno)

def check_vm_count(count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_vms(state)
    mycheck(len(s) == count, f'vm count {len(s)} == {count} for state {state}', lno)

    s=crm.get_vms('any')
    c=0
    for i in s:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms(state)
        for j in v:
            c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms('any')
        for j in v:
            if state != 'any' and j.state != state:
                continue
            c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)



def check_server_vm_count(sr, count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=fs.get_vms('any')
    c=0
    for i in v:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c==count, f'vm count in {sr} {c} == {count} for state={state}', lno) 
    if state == 'any':
        return
    v=fs.get_vms(state)
    c=0
    for i in v:
        c += 1
    mycheck(c==count, f'vm count in {sr} {c} == {count} for state={state}', lno) 

def check_server_vm_state(sr, vm, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=crm.get_vms('any')
    f=0
    for i in v:
        if i.vc.name == vm:
            f=1
            fv=i
            break
    if f != 1:
        mycheck(0, f'vm {vm} in {sr} lookup', lno)
    mycheck(fv.state==state, f'vm: {fv.vc.name} state {fv.state} == {state}', lno) 

def check_novm(vm):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_vms('any')
    c=0
    for i in s:
        if i.vc.name == vm:
            c += 1
    mycheck(c == 0, f'vm {vm} not present', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms('any')
        for j in v:
            if j.vc.name == vm:
                c += 1
    mycheck(c == 0, f'vm {vm} not present', lno)

def check_server_novm(sr, vm):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=crm.get_vms('any')
    f=0
    for i in v:
        if i.vc.name == vm:
            f=1
            fv=i
            break
    mycheck(f==0, f'vm: {vm}  in server {sr} not present', lno) 

aggr=namedtuple('aggr', 'ncpu mem')
vmt=namedtuple('vmt', 'ncpu mem')
testvm=namedtuple('testvm', 'vc pvm')

def get_available():
    available_cpus=0
    available_mem=0
    avails=[]
    s=crm.get_servers('any')
    for i in s:
        available_cpus += i.acpu
        available_mem += i.amem
        avails.append(sconf(i.sc.name, i.acpu, i.amem))

    print(avails)
    y=aggr(available_cpus, available_mem)
    return y
         

# 

# Last Test

In [2]:
global crm
crm=mycrm()
# 10 servers, 1000 vms
ps=[]
nservers=10
for i in range(0,nservers):
    sname=f's{i}'
    a = crm.add(sname, 150+i*50, 1000+2*i*200)
    ps.append(a)
print (f'Physical Servers: {ps}')
total_memory=0
total_cpu=0
for i in ps:
    total_memory +=i.sc.mem
    total_cpu += i.sc.ncpu
total = aggr(total_cpu,total_memory)
print (total)
t1=vmt(2, 4)
t2=vmt(4,16)
t3=vmt(8,64)
t4=vmt(16,128)
t5=vmt(32,256)
t6=vmt(64,512)
t7=vmt(128,1024)
vmtypes=[t1, t2, t3, t4, t5, t6, t7]
print(vmtypes)
# keep allocating until we run out of space
i=0
v=0
vms=[]
pvms=[]
mod=len(vmtypes) 
while True:
    vname=f'vm{v}'
    ncpu=vmtypes[i].ncpu
    mem=vmtypes[i].mem
    vm = crm.provision_vm(vname, ncpu, mem)
    if vm == None:
        print(f'provisioning failed for {vmtypes[i]}')
        mod -= 1
        if mod == 0:
            break
    else:
        tvm=testvm(vm.vc,vm)
        vms.append(tvm)
        pvms.append(vm)
        v += 1
    i=(i+1)%mod


allocated_cpus=0
allocated_mem=0
#    print(vms)
for v in pvms:
    allocated_cpus += v.vc.ncpu 
    allocated_mem += v.vc.mem 

allocated=aggr(allocated_cpus, allocated_mem)
available=get_available()
print(f'allocated:{allocated} Total: {total} available:{available}')
check_stat(crm_stat(stotal=nservers,sidle=nservers,sactive=0,sfail=0,vtotal=len(vms),vprovision=len(vms),vactive=0, vdeleted=0))

vmtypes=[t1, t2, t3, t4, t5, t6, t7]
for s in ps:
    check_server_state(s.sc.name, 'idle')
    check_vm_count(len(vms), 'provision')
    check_vm_count(len(vms), 'any')
    for v in s.get_vms('any'):
        check_server_vm_state(s.sc.name, v.vc.name, 'provision')
    check_server_vm_count(s.sc.name, s.nvms, 'provision')
    mycheck(((s.acpu < vmtypes[0].ncpu) or (s.amem < vmtypes[0].mem)),
            f'Server {s.sc.name, s.acpu, s.amem} available < min vm {vmtypes[0].ncpu}')

#undo last 10 operations

crm.provision_undo(10)
check_stat(crm_stat(stotal=nservers,sidle=nservers,sactive=0,sfail=0,vtotal=len(vms)-10,vprovision=len(vms)-10,vactive=0, vdeleted=10))

# verify that the last 10 vms are not present

for i in range(0,10):	
    v=vms[-i-1]
    pv=pvms[-i-1]
    check_novm(v.vc.name)
    mycheck((pv.state=='deleted'), f'vm {pv.vc.name} state {pv.state} == deleted')
#provision these vms again in the same order as before
for i in range(0,10):
    v=vms[len(vms)-10+i]
    print(f'recreating {v.vc.name}')
    vm = crm.provision_vm(v.vc.name, v.vc.ncpu, v.vc.mem)
    tvm=testvm(vm.vc,vm)
    vms[len(pvms)-10+i]=tvm
    pvms[len(pvms)-10+i]=vm
check_stat(crm_stat(stotal=nservers,sidle=nservers,sactive=0,sfail=0,vtotal=len(vms),vprovision=len(vms),vactive=0,
    vdeleted=10))


#now consume all cpus
v=len(vms)
print(v)
while True:
    vname=f'vm{v}'
    ncpu=1
    mem=1
    vm = crm.provision_vm(vname, ncpu, mem)
    if vm == None:
        print(f'provisioning failed for {vname, ncpu, mem}')
        break
    tvm=testvm(vm.vc,vm)
    vms.append(tvm)
    pvms.append(vm)
    v += 1
print(f'Provisioned total of {v} more vms')

allocated_cpus=0
allocated_mem=0
#    print(vms)
for v in pvms:
    allocated_cpus += v.vc.ncpu 
    allocated_mem += v.vc.mem 

allocated=aggr(allocated_cpus, allocated_mem)
available=get_available()
print(f'allocated:{allocated} Total: {total} available:{available}')
check_stat(crm_stat(stotal=nservers,sidle=nservers,sactive=0,sfail=0,vtotal=len(vms),vprovision=len(vms),vactive=0, vdeleted=10))

#commit all the vms

crm.commit_provisioned_vms()
check_stat(crm_stat(stotal=nservers,sidle=0,sactive=nservers,sfail=0,vtotal=len(vms),vprovision=0,vactive=len(vms), vdeleted=10))

# fail a server.
for p in ps:
    print(f'server: {p.sc.name}')

print(f'Failing few servers')
nvms=ps[0].nvms+ps[1].nvms
crm.fail(ps[0].sc.name)
crm.fail(ps[1].sc.name)
check_stat(crm_stat(stotal=nservers,sidle=0,sactive=nservers-2,sfail=2,vtotal=len(vms),vprovision=0,vactive=len(vms)-nvms,vidle=nvms, vdeleted=10))
idlevms=crm.get_vms('idle')
mycheck(len(idlevms)==nvms, f'idlevms count{len(idlevms)} match {nvms} vms in failed servers')




print(f'Deleting few servers')
crm.delete_server(ps[0].sc.name)
print(f'Deleting few servers')
crm.delete_server(ps[1].sc.name)
check_stat(crm_stat(stotal=nservers-2,sidle=0,sactive=nservers-2,sfail=0,sdeleted=2,vtotal=len(vms),vprovision=0,vactive=len(vms)-nvms,vidle=nvms, vdeleted=10))

#now delete some 10 vms
print(f'deleting 10 vms')
for i in range(0,9):
    crm.delete_vm(f'vm{i}')
# delete one of this idle vms.

idlevms=crm.get_vms('idle')
v1=idlevms[0]
print(f'deleting {v1.vc.name}')
crm.delete_vm(v1.vc.name)
nvms -= 1

# delete one idle vm

check_stat(crm_stat(stotal=nservers-2,sidle=0,sactive=nservers-2,sfail=0,sdeleted=2,vtotal=len(vms)-10,vprovision=0,vactive=len(vms)-nvms-10,vidle=nvms, vdeleted=20))

Physical Servers: [<myserver.myserver object at 0x7fbbf84f59a0>, <myserver.myserver object at 0x7fbbf84f5e80>, <myserver.myserver object at 0x7fbbf84f5fa0>, <myserver.myserver object at 0x7fbbf84f8100>, <myserver.myserver object at 0x7fbbf84f8250>, <myserver.myserver object at 0x7fbbf84f83a0>, <myserver.myserver object at 0x7fbbf84f84f0>, <myserver.myserver object at 0x7fbbf84f8640>, <myserver.myserver object at 0x7fbbf84f8790>, <myserver.myserver object at 0x7fbbf84f88e0>]
aggr(ncpu=3750, mem=28000)
[vmt(ncpu=2, mem=4), vmt(ncpu=4, mem=16), vmt(ncpu=8, mem=64), vmt(ncpu=16, mem=128), vmt(ncpu=32, mem=256), vmt(ncpu=64, mem=512), vmt(ncpu=128, mem=1024)]


AttributeError: 'mylist' object has no attribute 'myqueue'

In [3]:
for s in crm.server_list:
    for v in s.vl:
        print(v.name,v.state,s.name,s.state,v.acpu,v.amem)

vm36 active s9 active 4 16
vm35 active s9 active 2 4
vm29 active s9 active 4 16
vm28 active s9 active 2 4
vm23 active s9 active 8 64
vm22 active s9 active 4 16
vm21 active s9 active 2 4
vm18 active s9 active 32 256
vm17 active s9 active 16 128
vm16 active s9 active 8 64
vm15 active s9 active 4 16
vm14 active s9 active 2 4
vm13 active s9 active 128 1024
vm12 active s9 active 64 512
vm11 active s9 active 32 256
vm10 active s9 active 16 128
vm9 active s9 active 8 64
vm8 deleted s9 active 4 16
vm7 deleted s9 active 2 4
vm6 deleted s9 active 128 1024
vm5 deleted s9 active 64 512
vm4 deleted s9 active 32 256
vm3 deleted s9 active 16 128
vm2 deleted s9 active 8 64
vm1 deleted s9 active 4 16
vm0 deleted s9 active 2 4
vm50 active s8 active 4 16
vm49 active s8 active 2 4
vm44 active s8 active 8 64
vm43 active s8 active 4 16
vm42 active s8 active 2 4
vm38 active s8 active 16 128
vm37 active s8 active 8 64
vm32 active s8 active 32 256
vm31 active s8 active 16 128
vm30 active s8 active 8 64
vm27 ac

In [4]:
for s in crm.server_list:
    print(s.name,s.acpu,s.amem,s.nvms)

s9 264 2024 35
s8 18 0 16
s7 20 0 11
s6 22 0 11
s5 16 0 13
s4 18 0 11
s3 20 0 10
s2 22 0 5
s1 18 4 10
s0 18 0 8


In [5]:
#now reprovision the idle vms in those 10 vm slots.
c=crm.provision_all_idle_vms()
print(f'idle vms reprovisioned is {c}')
check_stat(crm_stat(stotal=nservers-2,sidle=0,sactive=nservers-2,sfail=0,sdeleted=2,vtotal=len(vms)-10,vprovision=0,vactive=len(vms)-nvms-10+c,vidle=nvms-c, vdeleted=20))


 Starting
s9 active 264 2024
s8 active 18 0
s7 active 20 0
s6 active 22 0
s5 active 16 0
s4 active 18 0
s3 active 20 0
s2 active 22 0
s1 deleted 18 4
s0 deleted 18 0
vconf(name='vm96', ncpu=64, mem=512)
s9 active 200 1512
s8 active 18 0
s7 active 20 0
s6 active 22 0
s5 active 16 0
s4 active 18 0
s3 active 20 0
s2 active 22 0
s1 deleted 18 4
s0 deleted 18 0

 Starting
s9 active 200 1512
s8 active 18 0
s7 active 20 0
s6 active 22 0
s5 active 16 0
s4 active 18 0
s3 active 20 0
s2 active 22 0
s1 deleted 18 4
s0 deleted 18 0
vconf(name='vm100', ncpu=32, mem=256)
s9 active 168 1256
s8 active 18 0
s7 active 20 0
s6 active 22 0
s5 active 16 0
s4 active 18 0
s3 active 20 0
s2 active 22 0
s1 deleted 18 4
s0 deleted 18 0

 Starting
s9 active 168 1256
s8 active 18 0
s7 active 20 0
s6 active 22 0
s5 active 16 0
s4 active 18 0
s3 active 20 0
s2 active 22 0
s1 deleted 18 4
s0 deleted 18 0
vconf(name='vm105', ncpu=8, mem=64)
s9 active 160 1192
s8 active 18 0
s7 active 20 0
s6 active 22 0
s5 active 16

In [13]:
for s in crm.server_list:
    print(s.name,s.state,s.acpu,s.amem)

s1 idle 200 1400
s9 active 4 0
s8 active 18 0
s7 active 20 0
s6 active 22 0
s5 active 16 0
s4 active 18 0
s3 active 20 0
s2 active 22 0
s0 idle 96 628


In [7]:
for i in crm.provision_list:
    print(i.name,i.amem,i.acpu,i.state)

In [8]:
for s in crm.server_list:
    print(s.name,s.state,s.acpu,s.amem)

s9 active 4 0
s8 active 18 0
s7 active 20 0
s6 active 22 0
s5 active 16 0
s4 active 18 0
s3 active 20 0
s2 active 22 0
s1 deleted 18 4
s0 deleted 18 0


In [9]:
vm=crm.get_vms('idle')
for v in vm:
    print(f'idlevm: {v.vc}')
print(f'Adding server config {ps[0].sc}')
a=crm.add(ps[0].sc.name, ps[0].sc.ncpu, ps[0].sc.mem)

idlevm: vconf(name='vm116', ncpu=2, mem=4)
idlevm: vconf(name='vm108', ncpu=4, mem=16)
idlevm: vconf(name='vm104', ncpu=4, mem=16)
idlevm: vconf(name='vm103', ncpu=16, mem=128)
idlevm: vconf(name='vm102', ncpu=8, mem=64)
idlevm: vconf(name='vm101', ncpu=4, mem=16)
idlevm: vconf(name='vm99', ncpu=16, mem=128)
Adding server config sconf(name='s0', ncpu=150, mem=1000)


In [10]:
ps[0]=a
print(f'adding provisioned {a.nvms} vms')

adding provisioned 7 vms


In [11]:
check_stat(crm_stat(stotal=nservers-1,sidle=1,sactive=nservers-2,sfail=0,sdeleted=2,vtotal=len(vms)-10,vprovision=0,vactive=len(vms)-nvms-10+c,vidle=nvms-c, vdeleted=20))

print(f'Adding server config {ps[1].sc}')
a=crm.add(ps[1].sc.name, ps[1].sc.ncpu, ps[1].sc.mem)
ps[1]=a
print(f'adding provisioned {a.nvms} vms')
check_stat(crm_stat(stotal=nservers,sidle=1,sactive=nservers-1,sfail=0,sdeleted=2,vtotal=len(vms)-10,vprovision=0,vactive=len(vms)-nvms-10+c+1,vidle=nvms-c-1, vdeleted=20))

# fail and unfail the servers. all vms should be provisioned exactly as before.

nvms1=ps[7].nvms
print(f'Failing server {ps[7].sc} nvms1 idle: {nvms1}')
crm.fail(ps[7].sc.name)
check_stat(crm_stat(stotal=nservers,sidle=1,sactive=nservers-2,sfail=1,sdeleted=2,vtotal=len(vms)-10,vprovision=0,vactive=len(vms)-nvms-10+c+1-nvms1,vidle=nvms-c-1+nvms1, vdeleted=20))
print(f'UnFailing server {ps[7].sc} nvms1 idle: {nvms1}')
crm.unfail(ps[7].sc.name)
pvms1=ps[7].nvms
if pvms1 != nvms1:
    print(f'fail/unfail did not reprovision all idle vms')
    print(f'faill vms: {nvms} reprovision vms: {pvms1}')
check_stat(crm_stat(stotal=nservers,sidle=1,sactive=nservers-1,sfail=0,sdeleted=2,vtotal=len(vms)-10,vprovision=0,vactive=len(vms)-nvms-10+c+1,vidle=nvms-c-1, vdeleted=20))

# now delete all physical servers

for p in ps:
    print(f'server: {p.sc.name}')

for p in ps:
    print(f'deleting {p.sc.name}')
    if p.nvms:
        te=0
        try:
            crm.delete_server(p.sc.name)
        except Exception as e:
            te=1
        s=f'active server {p.sc.name} should fail'
        mycheck(te==1, s)

# now delete all vms

#    for v in crm.get_vms('any'):
#        print(v.vc.name)
for v in vms:
    try:
        crm.delete_vm(v.vc.name)
    except Exception as e:
        s=f'Delete_vm returned exception for {v.vc}'
        print(e)

# check for zero vms
check_stat(crm_stat(stotal=nservers,sidle=nservers,sactive=0,sfail=0,sdeleted=2,vtotal=0,vprovision=0,vactive=0,vidle=0, vdeleted=130))

#delete all physical servers
for p in ps:
    print(f'deleting {p.sc.name}')
    mycheck(p.nvms==0, f'Server {p.sc} should have zero vms. Found {p.nvms}')
    try:
        crm.delete_server(p.sc.name)
    except Exception as e:
        s=f'Idle server {p.sc.name} can be deleted'
        mycheck(0, s)

check_stat(crm_stat(stotal=0,sidle=0,sactive=0,sfail=0,sdeleted=12,vtotal=0,vprovision=0,vactive=0,vidle=0, vdeleted=130))

1,41 tno: 291 crm_stat(stotal=9, sidle=1, sactive=8, sfail=0, sdeleted=2, vtotal=110, vprovision=0, vactive=103, vidle=7, vdeleted=20) == crm_stat(stotal=9, sidle=1, sactive=8, sfail=0, sdeleted=2, vtotal=110, vprovision=0, vactive=103, vidle=7, vdeleted=20) PASSED 
Adding server config sconf(name='s1', ncpu=200, mem=1400)
adding provisioned 0 vms
7,41: tno: 292 crm_stat(stotal=10, sidle=1, sactive=9, sfail=0, sdeleted=2, vtotal=110, vprovision=0, vactive=104, vidle=6, vdeleted=20) == crm_stat(stotal=10, sidle=2, sactive=8, sfail=0, sdeleted=2, vtotal=110, vprovision=0, vactive=103, vidle=7, vdeleted=20) FAILED 


SystemExit: 1

/home/blink/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import os
import sys
from stat import *
from mylist import *
from random import randrange


# cmp function for list lookups
# cmp node objects 
def node_obj_cmp(node, obj):
    if node.nobj == obj:
        return True
    else:
        return False

# compare object names

def node_objname_cmp(node, name):
    if node.nobjname == name:
        return True
    else:
        return False

def node_objid_cmp(node, id):
    if node.nobjid == id:
        return True
    else:
        return False

# file objects 
# mandatory to have these members.
# name - file name (not the full path)
# objid - file object is thisis the inode no. in the stat
# stat - stat object of the file returned by os.stat api.
# dir 0 (default) implies it is not directory but a regular file.
# dir 1 implies it is a directory.
# 

class myfile():

    # look above for argument descriptions.
    # raise exception if dir is 1 and not a dir
    # raise exception fi dir is 0 and not a regular file.
    # DO NOT CHANGE THIS. 
    # you can add additional members if required.

    def __init__(self, objid, name=None, stat=None, dir=0):

        # leave the following in there.
        if name and '/' in name:
            raise Exception("Bad name in file")
        if dir and stat and not S_ISDIR(stat.st_mode):
            s=f'{name} is not a directory'
            raise Exception(s)
        if not dir and stat and not S_ISREG(stat.st_mode):
            s=f'{name} is not a file'
            raise Exception(s)

        self.name=name
        self.objid = objid
        self.stat=stat
        self.dir=dir
        #YOUR CODE
        #raise NotImplementedError

    
    # return the file path from root of the tree to which it belongs.
    # eg., I will create a file object with
    # fobj=myfile(241573) ie. only with objectid.
    # print(fobj.path(tree))
    # where this file is part of the tree.  Note: not this instance
    # of the object but the same objectid.
    # it should print the file path.
    # the objectid is found by "ls -i1" in any of the subdirectory
    # of the kernel-0 folder sent to you.  you can do that even on windows
    # with powershell i believe.
    # this function should print the full path of that objectid starting
    # from the kernel-0 folder.
    # eg., if the file kernel-0/mm/f1.c and its objectid is 241573
    # then fobj.file_path(tree) should print kernel-0/mm/f1.c
    # Note: this has to be recalculated from the tree when this function
    # is called. Tree was the object obtained from convert_to tree function
    # at the end of this file.


    def file_path(self, tree):
        pass
        #YOUR CODE
        #raise NotImplementedError

    # return the file size
    # required for the binary tree organization of these file objects.
    # for assignment 6.
    #

    def file_size(self):
        pass
        #YOUR CODE
        #raise NotImplementedError

    # this is just a helper. feel free to modify the way you want it to help
    # you.

    def __str__(self):
        return self.name

# directory objects inherit from file, uses 
# all of what file has plus some more interfaces.


class mydir(myfile):

    # DO NOT CHANGE THIS.
    # you can add any members below this.

    def __init__(self, objid, name=None, stat=None, ):
        super().__init__(objid, name, stat, 1)
        #YOUR CODE
        #raise NotImplementedError

    # find the total no. of files underneath this directory including 
    # the directories (as they are also files) as well. 
    # eg., if you have a folder : kernel-0/mm/f1.c and kernel-0/mm/f2.c
    # dir_total_files of the directory kernel-0 will print : 3. two regular
    # files and one directory file "mm".
    # use case will be :
    # dir=mydir(245173)
    # dir.dir_total_files(tree)
    # where this directory is part of the tree.  Note: not this instance
    # of the object but the same objectid.
    # Note this has to be "calculated" every time the function is called
    # from the passed tree. 

    def dir_total_files(self, tree):
        #YOUR CODE
        #raise NotImplementedError
        pass

    # the below function is similar as above but should print the only the total
    # no. of directories underneath this directory.
    # Note this has to be "calculated" every time the function is called
    # from the passed tree. 

    def dir_total_dirs(self, tree):
        pass
        #YOUR CODE
        raise NotImplementedError


class mynode():
    # pobj - parent object
    # obj - this object
    # objname - name of this object
    # objid - unique id of this object. for file and dir it is inode numbes.
    # Do not change this.

    def __init__(self, pobj, obj, objname, objid):
        self.child_list=mylist("list")
        self.nobj=obj
        self.nobjname = objname
        self.nobjid=objid
        self.npobj = pobj

class mytree():

# init functionf or root object and name and its object id.
# (object id for these objects are all inode numbers).
# Do not change this.

    def __init__(self, obj, root_name, id):
        self.root=mynode(None, obj, root_name, id)
        self.total_nodes=1

# the following two functions are helper functions for 
# constructing the tree, insertions etc..


# Lookup objname from a parent node, if found return
# node object. 
# If it does not  exist, create a node for that objname,
# objid combinations, ensure to add it to the appropriate child
# list.

    def lookup_create(self, pobj:mynode, objname, objid, obj):
        #YOUR CODE
        #raise NotImplementedError

# Lookup objname from a parent node, if found return
# node object. If not, return None.

    def lookup(self, pobj:mynode, objname):
        #YOUR CODE
        #raise NotImplementedError


# Method on the tree object to traverse and it will call the function
# func with the passed arguments (variable argument args), for 
# every line of the object it will generate as output corresponding 
# to the input file in function convert_input_to_tree().
#
# for the eg. mentioned in the comments of that function
# the function func() will get called for each of the
# lines given there. And that string will be passed as first argument to the
# function.

# func_traverse(tree, test_func, argobj) returned by convert_input_to_tree() will call 
# test_func(fobj, "kernel-0", argobj) where fobj is the mydir obj for kernel-0.
# test_func(fobj, "kernel-0/arch", argobj) where fobj is the mydir obj for kernel-0/arch.
# test_func(fobj, "kernel-0/arch/boot", argobj) where fobj is the mydir obj for kernel-0/arch/boot.
# test_func(fobj, "kernel-0/arch/boot/bootloader.lds", argobj) where fobj is the  myfile obj 
# for kernel-0/arch/boot/bootloader.lds.

# you can test this function by putting the string in some file or std output
# and then compare the entire output with the original input file (sort both to
# avoid any ordering issues).

    def func_traverse(self, func, *args):
        #YOUR CODE
        #raise NotImplementedError
        pass

# exactly same as above function except that the traversal starts
# from an intermediary mynode() object ie., an interior node.

    def func_node_traverse(self, node, func, *args):
        #YOUR CODE
        #raise NotImplementedError
        pass

# some debug stuff may be useful.

    def dump1(self, root):
        print(root.obj)
        print(f'{root.obj} Child: ', end="")
        for i in root.child_list:
            print(f'{i.obj}, ', end="")
        print('\n')
        for i in root.child_list:
            self.dump1(i)

# some debug stuff may be useful.

    def dump(self):
        print(f'Total Nodes: {self.total_nodes}')
        self.dump1(self.root)


# Function takes an input file name 
# and converts that into an object instance of 
# the tree class as defined above.
# you can only use the operations given above to construct the tree.
# no other functions should be added.
# input file will be of the form:
#kernel-0
#kernel-0/arch
#kernel-0/arch/alpha
#kernel-0/arch/alpha/boot
#kernel-0/arch/alpha/boot/bootloader.lds
# the above should be converted to tree with root object being the
# directory object - kernel-0, and there are arch, alpha and boot as
# directory objects below and bootloader.lds as a file object below.
# construct the appropriate hierarchy. The mynode.obj should be 
# either a mydir or myfile object as defined above.

# returns "tree" object.

# hint use readlines, strip, and split if required  and os.stat

def convert_input_to_tree(filename):
    pass
    #YOUR CODE
    #raise NotImplementedError


## Crm test

In [ ]:
global crm
crm=mycrm()
a = crm.add("s1", 500, 1000)
print(a.sc)
check_server_count(1, 'any')
s=crm.get_servers('any')
mycheck((len(s) == 1), "get servers should return one entry")
check_server_count(1, 'idle')
check_server_state('s1', 'idle')
s=crm.get_stat()
check_stat(crm_stat(stotal=1,sidle=1))
crm.fail(a.sc.name)
check_stat(crm_stat(stotal=1,sfail=1))
check_server_state('s1', 'fail')
crm.unfail(a.sc.name)
check_stat(crm_stat(stotal=1,sidle=1))
check_server_state('s1', 'idle')
crm.delete_server(a.sc.name)
check_stat(crm_stat(sdeleted=1))
check_server_count(0, 'any')

print(f'*********crm_1server tests done *********')
print(f'*********crm_1server tests done *********')

# CRMVM test


In [ ]:
import os
import sys
from mylist import *
from mycrm import *
from myserver import *
from myvm import *
from inspect import currentframe, getframeinfo

# test.py for toy cloud assignment.
# feel free to comment the relevant parts in main function.

## test helper functions

global test_no
global crm
test_no=0
def mycheck(result, msg, lno=0):
    global test_no
    test_no += 1
    cf=currentframe()
    l1=cf.f_back.f_lineno
    if lno:
        l=f'{lno},{l1}'
    else:
        l=f'{l1}'
    if result:
        print(f'{l} tno: {test_no} {msg} PASSED ')
    else:
        print(f'{l}: tno: {test_no} {msg} FAILED ')
        sys.exit(1)

# template crmstat to copy and change values.
#mys=crm_stat(stotal=1,sidle=1,sactive=0,sfail=0,vtotal=1,vprovision=1,vactive=0,'vidle', vdeleted=0)
#print(mys)


def check_stat(s1):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_stat()
    mycheck((s==s1),f'{s1} == {s}', lno)

def check_server_count(count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_servers(state)
    mycheck(len(s) == count, f'server_count {len(s)} == {count} for state {state}', lno)

    s=crm.get_servers('any')
    c=0
    for i in s:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c == count, f'server_count {c} == {count} for state {state}', lno)

def check_server_state(sname, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    assert(state != 'any')
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sname and i.state == state:
            f=1
            break
    mycheck(f==1, f'server {sname} with state {state} found', lno)
    s=crm.get_servers(state)
    f=0
    for i in s:
        if i.sc.name == sname and i.state == state:
            f=1
            break
    mycheck(f==1, f'server {sname} with state {state} found', lno)

def check_vm_count(count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_vms(state)
    mycheck(len(s) == count, f'vm count {len(s)} == {count} for state {state}', lno)

    s=crm.get_vms('any')
    c=0
    for i in s:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms(state)
        for j in v:
            c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms('any')
        for j in v:
            if state != 'any' and j.state != state:
                continue
            c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)



def check_server_vm_count(sr, count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=fs.get_vms('any')
    c=0
    for i in v:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c==count, f'vm count in {sr} {c} == {count} for state={state}', lno) 
    if state == 'any':
        return
    v=fs.get_vms(state)
    c=0
    for i in v:
        c += 1
    mycheck(c==count, f'vm count in {sr} {c} == {count} for state={state}', lno) 

def check_server_vm_state(sr, vm, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)
    v=crm.get_vms('any')
    f=0
    for i in v:
        if i.vc.name == vm:
            f=1
            fv=i
            break
    if f != 1:
        mycheck(0, f'vm {vm} in {sr} lookup', lno)
    mycheck(fv.state==state, f'vm: {fv.vc.name} state {fv.state} == {state}', lno) 

def check_novm(vm):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_vms('any')
    c=0
    for i in s:
        if i.vc.name == vm:
            c += 1
    mycheck(c == 0, f'vm {vm} not present', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms('any')
        for j in v:
            print(j.vc.name,j.state)
            if j.vc.name == vm:
                c += 1
    mycheck(c == 0, f'vm {vm} not present', lno)

def check_server_novm(sr, vm):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=crm.get_vms('any')
    f=0
    for i in v:
        if i.vc.name == vm:
            f=1
            fv=i
            break
    mycheck(f==0, f'vm: {vm}  in server {sr} not present', lno) 

global crm
crm=mycrm()
a = crm.add("s1", 500, 1000)
print(a)
check_server_state("s1", 'idle')
v=crm.provision_vm("vm1", 2, 50)
print(v)
check_server_state("s1", 'idle')
check_stat(crm_stat(stotal=1,sidle=1,vtotal=1,vprovision=1))
check_vm_count(1, 'provision')
check_vm_count(1, 'any')
check_server_vm_state("s1", "vm1", 'provision')
check_server_vm_count("s1", 1, 'provision')
crm.commit_provisioned_vms()
check_server_state("s1", 'active')
check_server_vm_count("s1", 1, 'active')
check_server_vm_state("s1", "vm1", 'active')
check_stat(crm_stat(stotal=1,sactive=1,vtotal=1,vactive=1))
check_vm_count(1, 'any')
check_vm_count(1, 'active')
te=0
try:
    crm.delete_server("s1")
except Exception as e:
    te=1
mycheck(te == 1, 'Cannot delete an active server')

# provision one more vm and then undo it.
v=crm.provision_vm("vm2", 2, 50)
print(v)
check_stat(crm_stat(stotal=1,sactive=1,vtotal=2,vprovision=1,vactive=1))
check_vm_count(2, 'any')
check_vm_count(1, 'active')
check_vm_count(1, 'provision')
check_server_vm_count("s1", 1, 'active')
check_server_vm_state("s1", "vm1", 'active')
check_server_vm_count("s1", 1, 'provision')
check_server_vm_state("s1", "vm2", 'provision')

In [ ]:
for i in crm.get_vms('any'):
    print(i.name,i.state)

In [ ]:
crm.provision_undo(1)
check_stat(crm_stat(stotal=1,sactive=1,vtotal=1,vactive=1,vdeleted=1))

In [ ]:
for i in crm.get_vms('any'):
    print(i.name,i.state)

In [ ]:
check_vm_count(1, 'any')

In [ ]:
for i in crm.get_vms('any'):
    print(i.name,i.state)

In [ ]:
count=1
state='any'

cf=currentframe()
lno=cf.f_back.f_lineno
s=crm.get_vms(state)
mycheck(len(s) == count, f'vm count {len(s)} == {count} for state {state}', lno)

s=crm.get_vms('any')
c=0
for i in s:
    if state != 'any' and i.state != state:
        continue
    c += 1
mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
c=0
s=crm.get_servers('any')
for i in s:
    v=i.get_vms(state)
    for j in v:
        c += 1
mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
c=0
s=crm.get_servers('any')
for i in s:
    v=i.get_vms('any')
    for j in v:
        if state != 'any' and j.state != state:
            continue
        c += 1
mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)


In [ ]:

check_server_vm_count("s1", 1, 'any')
check_server_vm_count("s1", 1, 'active')
check_novm("vm2")
check_server_novm("s1", "vm2")
te=0
try:
    crm.delete_vm('random')
except Exception as e:
    te=1
mycheck(te == 1, 'Wrong name to delete')
crm.delete_vm("vm1")
check_vm_count(0, 'any')
check_server_vm_count("s1", 0, 'any')
check_novm("vm1")
check_server_novm("s1", "vm1")
check_stat(crm_stat(stotal=1,sidle=1,vdeleted=2))
crm.delete_server(a.sc.name)
check_stat(crm_stat(vdeleted=2,sdeleted=1))
check_server_count(0, 'any')
check_vm_count(0, 'any')
print(f'********crm_1vm tests done ************')
print(f'********crm_1vm tests done ************')


# VM Test

In [ ]:

import sys
from mylist import *
from mycrm import *
from myserver import *
from myvm import *

from inspect import currentframe, getframeinfo

# test.py for toy cloud assignment.
# feel free to comment the relevant parts in main function.

## test helper functions

global test_no
global crm
test_no=0
def mycheck(result, msg, lno=0):
    global test_no
    test_no += 1
    cf=currentframe()
    l1=cf.f_back.f_lineno
    if lno:
        l=f'{lno},{l1}'
    else:
        l=f'{l1}'
    if result:
        print(f'{l} tno: {test_no} {msg} PASSED ')
    else:
        print(f'{l}: tno: {test_no} {msg} FAILED ')
        sys.exit(1)

# template crmstat to copy and change values.
#mys=crm_stat(stotal=1,sidle=1,sactive=0,sfail=0,vtotal=1,vprovision=1,vactive=0,'vidle', vdeleted=0)
#print(mys)


def check_stat(s1):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_stat()
    mycheck((s==s1),f'{s1} == {s}', lno)

def check_server_count(count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_servers(state)
    mycheck(len(s) == count, f'server_count {len(s)} == {count} for state {state}', lno)

    s=crm.get_servers('any')
    c=0
    for i in s:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c == count, f'server_count {c} == {count} for state {state}', lno)

def check_server_state(sname, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    assert(state != 'any')
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sname and i.state == state:
            f=1
            break
    mycheck(f==1, f'server {sname} with state {state} found', lno)
    s=crm.get_servers(state)
    f=0
    for i in s:
        if i.sc.name == sname and i.state == state:
            f=1
            break
    mycheck(f==1, f'server {sname} with state {state} found', lno)

def check_vm_count(count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_vms(state)
    mycheck(len(s) == count, f'vm count {len(s)} == {count} for state {state}', lno)

    s=crm.get_vms('any')
    c=0
    for i in s:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms(state)
        for j in v:
            c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms('any')
        for j in v:
            if state != 'any' and j.state != state:
                continue
            c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)



def check_server_vm_count(sr, count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=fs.get_vms('any')
    c=0
    for i in v:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c==count, f'vm count in {sr} {c} == {count} for state={state}', lno) 
    if state == 'any':
        return
    v=fs.get_vms(state)
    c=0
    for i in v:
        c += 1
    mycheck(c==count, f'vm count in {sr} {c} == {count} for state={state}', lno) 

def check_server_vm_state(sr, vm, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=crm.get_vms('any')
    f=0
    for i in v:
        if i.vc.name == vm:
            f=1
            fv=i
            break
    if f != 1:
        mycheck(0, f'vm {vm} in {sr} lookup', lno)
    mycheck(fv.state==state, f'vm: {fv.vc.name} state {fv.state} == {state}', lno) 

def check_novm(vm):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_vms('any')
    c=0
    for i in s:
        if i.vc.name == vm:
            c += 1
    mycheck(c == 0, f'vm {vm} not present', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms('any')
        for j in v:
            if j.vc.name == vm:
                c += 1
    mycheck(c == 0, f'vm {vm} not present', lno)

def check_server_novm(sr, vm):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=crm.get_vms('any')
    f=0
    for i in v:
        if i.vc.name == vm:
            f=1
            fv=i
            break
    mycheck(f==0, f'vm: {vm}  in server {sr} not present', lno) 

aggr=namedtuple('aggr', 'ncpu mem')
vmt=namedtuple('vmt', 'ncpu mem')
testvm=namedtuple('testvm', 'vc pvm')

def get_available():
    available_cpus=0
    available_mem=0
    avails=[]
    s=crm.get_servers('any')
    for i in s:
        available_cpus += i.acpu
        available_mem += i.amem
        avails.append(sconf(i.sc.name, i.acpu, i.amem))

    print(avails)
    y=aggr(available_cpus, available_mem)
    return y
         
    
global crm
crm=mycrm()
a = crm.add("s1", 500, 1000)
print(a.sc)
check_server_count(1, 'any')
s=crm.get_servers('any')
mycheck((len(s) == 1), "get servers should return one entry")
check_server_count(1, 'idle')
check_server_state('s1', 'idle')
s=crm.get_stat()
check_stat(crm_stat(stotal=1,sidle=1))
crm.fail(a.sc.name)
check_stat(crm_stat(stotal=1,sfail=1))
check_server_state('s1', 'fail')
crm.unfail(a.sc.name)
check_stat(crm_stat(stotal=1,sidle=1))
check_server_state('s1', 'idle')
crm.delete_server(a.sc.name)
check_stat(crm_stat(sdeleted=1))
check_server_count(0, 'any')

print(f'*********crm_1server tests done *********')
print(f'*********crm_1server tests done *********')
